In [1]:
# All  import statements needed for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import *

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# **Load Model and Predict**
MODEL = '/content/drive/MyDrive/models/model_split_softmax_cnn_model.hdf5'
from tensorflow import keras
model = keras.models.load_model(MODEL)

In [5]:
data = pd.read_csv('/content/drive/MyDrive/balancedhalf_data.csv')
data.drop(data.columns[0] , inplace=True , axis=1)
unknown = ['SSH-Patator','DoS slowloris','DoS Slowhttptest','Bot','Infiltration','Heartbleed']
att = data.loc[(data['Label'].isin(unknown))]
data.drop(att.index,axis=0 , inplace=True, errors='ignore')
# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns
# using standard scaler for normalizing
std_scaler = MinMaxScaler()
def normalization(df,att,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    x = np.array(att[i])
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    #To use the same scaler which was used in preprocessing the train data 
    att[i] = std_scaler.transform(x.reshape(len(x),1))
  return df,att
# calling the normalization() function
data , att = normalization(data.copy(),att.copy(),numeric_col)
att.Label = 'unknown'
X = att.drop('Label' , axis=1)
X = X.to_numpy().reshape(-1, 83,1)

y = att.Label
X_train = data.drop('Label' , axis=1)
X_train = X_train.to_numpy().reshape(-1, 83,1)

y_train = data.Label

In [6]:
predicted = model.predict(X)
Y_train_predicted = model.predict(X_train)

34117/34117 [==============================] - 80s 2ms/step


In [7]:
my_dict = {
    0 : 'BENIGN',
    1 : 'DDoS',
    2 : 'DoS Hulk',
    3 : 'DoS GoldenEye',
    4 : 'PortScan',
    5 : 'FTP-Patator'
}
max_probability = np.amax(predicted, axis = 1)
unknown_index = np.argmax(predicted, axis = 1)
unknown_index_label = np.vectorize(my_dict.get)(unknown_index)
max_probability_train = np.amax(Y_train_predicted, axis = 1)
y_prediction_index = np.argmax(Y_train_predicted, axis = 1)
y_prediction_test_label = np.vectorize(my_dict.get)(y_prediction_index)
RIGHT_BOOLEAN = y_train == y_prediction_test_label

In [8]:
threshold_values = {
    'BENIGN': 0.3499285,
    'DDoS': 1.0,
    'DoS GoldenEye': 0.9743001,
    'DoS Hulk': 0.7003728,
    'FTP-Patator': 0.9999999,
    'PortScan': 0.9105322
 }
RIGHT_BOOLEAN = RIGHT_BOOLEAN.to_numpy()

In [9]:
print('THE THRESHOLDS ARE : ',threshold_values)

THE THRESHOLDS ARE :  {'BENIGN': 0.3499285, 'DDoS': 1.0, 'DoS GoldenEye': 0.9743001, 'DoS Hulk': 0.7003728, 'FTP-Patator': 0.9999999, 'PortScan': 0.9105322}


In [10]:
TOTAL = data.shape[0]+att.shape[0]
## WITH BOTH TRAIN TEST && UNKNOWN SET
truepredicted = 0
# 1.0 TRAIN SET:
for i in range(len(y_prediction_test_label)):
  threshold = threshold_values[y_prediction_test_label[i]]
  prob = max_probability_train[i]
  ##check max_prob > threshold and prediction is correct
  if prob > threshold and RIGHT_BOOLEAN[i] :
    truepredicted += 1

# 2.0 UNKNOWN SET:
for i in range(len(unknown_index_label)):
  threshold = threshold_values[unknown_index_label[i]]
  prob = max_probability[i]
  ## check if prob<threshold, then right prediction
  if prob < threshold:
    truepredicted += 1

## ACCURACY PREDICTION:
Accuracy = truepredicted/TOTAL
print('Accuracy is ',Accuracy)

Accuracy is  0.5040470523848445


In [11]:
truepredicted

559962

In [12]:
TOTAL

1110932

In [13]:
TOTAL = data.shape[0]
## WITH TRAIN TEST
truepredicted = 0
# 1.0 TRAIN SET:
for i in range(len(y_prediction_test_label)):
  threshold = threshold_values[y_prediction_test_label[i]]
  prob = max_probability_train[i]
  ##check max_prob > threshold and prediction is correct
  if prob > threshold and RIGHT_BOOLEAN[i] :
    truepredicted += 1


## ACCURACY PREDICTION:
Accuracy = truepredicted/TOTAL
print('Accuracy is ',Accuracy)

Accuracy is  0.5083157236195496


In [14]:
TOTAL = att.shape[0]
## WITH UNKNOWN SET
truepredicted = 0

# 2.0 UNKNOWN SET:
for i in range(len(unknown_index_label)):
  threshold = threshold_values[unknown_index_label[i]]
  prob = max_probability[i]
  ## check if prob<threshold, then right prediction
  if prob < threshold:
    truepredicted += 1

## ACCURACY PREDICTION:
Accuracy = truepredicted/TOTAL
print('Accuracy is ',Accuracy)

Accuracy is  0.2613902629523562


In [15]:
data.shape

(1091727, 84)

In [16]:
att.shape

(19205, 84)

In [17]:
TOTAL = att.shape[0]
## WITH all SET
truepredicted = 0

# 1.0 TRAIN SET:
for i in range(len(y_prediction_test_label)):
  threshold = threshold_values[y_prediction_test_label[i]]
  prob = max_probability_train[i]
  ##check max_prob > threshold and prediction is correct
  if prob > threshold and RIGHT_BOOLEAN[i] :
    truepredicted += 1

# 2.0 UNKNOWN SET:
for i in range(len(unknown_index_label)):
  threshold = threshold_values[unknown_index_label[i]]
  prob = max_probability[i]
  ## check if prob<threshold, then right prediction
  if prob < threshold:
    truepredicted += 1

## ACCURACY PREDICTION:
Accuracy = truepredicted/TOTAL
print('Accuracy is ',Accuracy)

Accuracy is  29.157094506638895
